## Solving bidomain equations on the slice with the Beeler-Reuter model and ischemia

### Importing required libraries

In [1]:
#%cd ../../..
from src.models.dynamics_models import *
from src.models.cell_models import *
from src.utils import *
import ufl
%pwd

'/home/drvinko/Desktop/Modeliranje-biofizike-srcanog-tkiva'

### Defining a domain, cell model and dynamics model and visualizing initial data

In [2]:
domain = heart_slice()
x = ufl.SpatialCoordinate(domain)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Nurb)
Info    : [ 40%] Meshing curve 2 (Nurb)
Info    : [ 70%] Meshing curve 3 (Nurb)
Info    : Done meshing 1D (Wall 0.0263991s, CPU 0.025253s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0618841s, CPU 0.061774s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 2.2515e-05s, CPU 2.3e-05s)
Info    : 3628 nodes 7231 elements


In [3]:
class CellModel(ReparametrizedFitzHughNagumo):
    def apply_current(self):
        locator = lambda x: (x[0] - 4.3) ** 2 + (x[1] - 1.7) ** 2 < 0.3**2
        cells = fem.locate_dofs_geometrical(self.V1, locator)
        self.I_app.x.array[cells] = np.full_like(cells, 10)
        self.I_app_duration = 20

In [4]:
class Model(BidomainModel):
    def initial_V_m(self):
        self.V_m_n.x.array[:] = -85

    def conductivity(self):
        # Muscle fibres
        x_c, y_c = 0.2, 1
        self.fibers = ufl.as_vector(
            [
                -(x[1] - y_c) / ufl.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
                (x[0] - x_c) / ufl.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
            ]
        )

        # Healthy conductivities
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IL - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)
        self.M_e = self.SIGMA_ET * ufl.Identity(2) + (
            self.SIGMA_EL - self.SIGMA_ET
        ) * ufl.outer(self.fibers, self.fibers)

    def ischemia(self):
        x_c, y_c = 4.5, 2.8
        locator = lambda x: (x[0] - x_c) ** 2/10 + (x[1] - y_c)**2*10 < 0.5**2
        self.M_i = ufl.conditional(
            locator(self.x),
            self.M_i / 10,
            self.M_i,
        )
        self.M_e = ufl.conditional(
            locator(self.x),
            self.M_e / 15,
            self.M_e,
        )
        return locator

In [5]:
cell_model = CellModel(domain)
model = Model(domain, cell_model)

In [6]:
cell_model.plot_I_app(
    show_mesh=False,
    function_name="",
    camera_direction="xy",
    save_to="figures/slice/ischemia_MFN_applied_current.pdf",
)

Widget(value='<iframe src="http://localhost:36033/index.html?ui=P_0x7efbc98b4910_0&reconnect=auto" class="pyvi…

In [7]:
model.plot_ischemia(
    show_mesh=False,
    function_name="",
    camera_direction="xy",
    save_to="figures/slice/ischemia_MFN_ischemic_area.pdf",
)

Widget(value='<iframe src="http://localhost:36033/index.html?ui=P_0x7efb827c0710_1&reconnect=auto" class="pyvi…

### Solving equations with given parameters

In [8]:
model.solve(
    T=500,
    steps=500,
    save_to="slice/ischemia_MFN.mp4",
    camera_direction="xy",
)

Solving problem: 100%|██████████| 500/500 [05:31<00:00,  1.51it/s]


### Plotting a fiber orientations

In [9]:
x_c, y_c = 0.2, 1
plot_vector_field(
    domain,
    lambda x: (
        -(x[1] - y_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        (x[0] - x_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        0,
    ),
    0.03,
    0.3,
    camera_direction="xy",
    save_to="figures/slice/ischemia_MFN_fibers.pdf",
)

Widget(value='<iframe src="http://localhost:36033/index.html?ui=P_0x7efbd8e95b10_2&reconnect=auto" class="pyvi…

### Plotting a final plot of transmembrane potential

In [10]:
plot_function(model.V_m_n, "V_m", show_mesh=False)

Widget(value='<iframe src="http://localhost:36033/index.html?ui=P_0x7efb74337b50_3&reconnect=auto" class="pyvi…

### Defining a new model

In [11]:
class NewModel(BidomainModel):
    def initial_V_m(self):
        self.V_m_n.x.array[:] = model.V_m_n.x.array[:]

    def conductivity(self):
        # Muscle fibres
        x_c, y_c = 0.2, 1
        self.fibers = ufl.as_vector(
            [
                -(x[1] - y_c) / ufl.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
                (x[0] - x_c) / ufl.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
            ]
        )

        # Healthy conductivities
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IL - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)
        self.M_e = self.SIGMA_ET * ufl.Identity(2) + (
            self.SIGMA_EL - self.SIGMA_ET
        ) * ufl.outer(self.fibers, self.fibers)

    def ischemia(self):
        x_c, y_c = 2.0, 1.5
        locator = lambda x: (x[0] - x_c) ** 2/10 + (x[1] - y_c)**2*10 < 0.5**2
        self.M_i = ufl.conditional(
            locator(self.x),
            self.M_i / 10,
            self.M_i,
        )
        self.M_e = ufl.conditional(
            locator(self.x),
            self.M_e / 15,
            self.M_e,
        )
        return locator

In [12]:
new_cell_model = ReparametrizedFitzHughNagumo(domain)
new_model = NewModel(domain, new_cell_model)

In [13]:
new_model.plot_initial_V_m(
    show_mesh=False,
    function_name="",
    camera_direction="xy",
    save_to="figures/slice/ischemia_MFN_initial_V_m.pdf",
)

Widget(value='<iframe src="http://localhost:36033/index.html?ui=P_0x7efbd8f0e310_4&reconnect=auto" class="pyvi…

In [14]:
new_model.plot_ischemia(
    show_mesh=False,
    function_name="",
    camera_direction="xy",
    save_to="figures/slice/ischemia_MFN_ischemic_area.pdf",
)

Widget(value='<iframe src="http://localhost:36033/index.html?ui=P_0x7efbd8f0d050_5&reconnect=auto" class="pyvi…

### Solving equations with given parameters

In [15]:
new_model.solve(
    T=2000,
    steps=2000,
    save_to="slice/ischemia_MFN.mp4",
    camera_direction="xy",
    checkpoints=[200, 500, 900, 1300, 1500],
    checkpoint_file="figures/slice/ischemia_MFN",
)

Solving problem: 100%|██████████| 2000/2000 [28:21<00:00,  1.18it/s]


ERROR:wslink.protocol:Exception raised
ERROR:wslink.protocol:KeyError('f334a46714153d0bc1be69fb680f7c47_13957d')
ERROR:wslink.protocol:Traceback (most recent call last):
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/wslink/protocol.py", line 332, in onMessage
    results = func(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/protocols/local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/serializers/synchronization_context.py", line 30, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
                ~~~~~~~~~~~~~~~~~~~~~^^^^^^^
KeyError: 'f334a46714153d0bc1be69fb680f7c47_13957d'

ERROR:wslink.protocol:Exception raised
ERROR:wslink.protocol:KeyError('2746aea41ec18e64987f391cabc10da7_13957d